### Setup environment
Notebook uses OpenAI API. Use an existing key or [create a new key](https://platform.openai.com/account/api-keys).

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

### Setup logger

In [2]:
import sys
import logging

# Load logging level from environment or set default level to ERROR
env_logging_level = os.environ.get("LOGGING_LEVEL", "ERROR")
level = getattr(logging, env_logging_level.upper(), logging.ERROR)

logging.basicConfig(stream=sys.stdout, level=level)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

### Import dependence 

In [3]:
from pathlib import Path
from IPython.display import Markdown
from llama_index import LLMPredictor, ServiceContext, download_loader, VectorStoreIndex
from langchain import OpenAI

INFO:numexpr.utils:Note: NumExpr detected 10 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
Note: NumExpr detected 10 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
NumExpr defaulting to 8 threads.


### Import Llama hub dependence
Import loaders for PDF and Markdown.

In [4]:
PDFMinerReader = download_loader("PDFMinerReader")
pdf_loader = PDFMinerReader()

MarkdownReader = download_loader("MarkdownReader")
md_loader = MarkdownReader()

### Setup LLM, vector store and context

In [5]:
# load index
llm_predictor = LLMPredictor(llm=OpenAI(temperature=0, model_name="text-davinci-003"))
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor)

In [9]:
file_path = os.environ.get("PLANNING_PERMISSION_FILE_PATH")

if (file_path == None):
    raise Exception("PLANNING_PERMISSION_FILE_PATH not set")

if file_path.endswith(".pdf"):
    document = pdf_loader.load_data(file=Path(file_path))
elif file_path.endswith(".md"):
    document = md_loader.load_data(file=Path(file_path))
else:
    raise Exception("File format not supported")

index = VectorStoreIndex.from_documents(document, service_context=service_context)

query_engine = index.as_query_engine()

### Query single question
Example of how to manually query a question.

Remove (or comment) `%%script echo Skipping cell` before running cell!

In [7]:
# %%script echo Skipping cell

response = query_engine.query("Behöver jag söka bygglov för en altan?")
display(Markdown(f"<b>{response}</b>"))

<b>
Ja, du behöver söka bygglov för en altan om den är högre än 1,8 meter, placeras mindre än 4,5 meter från gränsen, eller om den kan betraktas som en volymökning eller väsentligt ändrar fasaden.</b>

### Questions can be loaded from a JSON file
Set optinal environment variable `PLANNING_PERMISSION_QUESTIONS_FILE_PATH=./data/questions.json` to load questions from file.

Example file with questions can be find in folder `example_data`.

For this cell we measure CPU and wall runtime by setting `%%time`.

Example of JSON file:
```json
{
    "questions": [
        {
            "id": "q1",
            "question": "Måste man söka bygglov för flaggstång?"
        },
        {
            "id": "q2",
            "question": "Måste man söka bygglov för altan?"
        }
    ]
}
```

In [8]:
%%time

import json

file_path = os.environ.get("PLANNING_PERMISSION_QUESTIONS_FILE_PATH")

if file_path is not None:
    try:
        with open(file_path) as f:
            data = json.load(f)
            questions = data.get("questions", [])
            
            for question in questions:
                query = question.get("question")
                print(f"User: {json.dumps(query, ensure_ascii=False)}")

                response = query_engine.query(query)
                display(Markdown(f"Assistant: <b>{response}</b>"))
    
    except FileNotFoundError:
        print(f"The file {file_path} does not exist.")
    except json.JSONDecodeError:
        print(f"The file {file_path} is not in a valid JSON format.")
    except Exception as e:
        import traceback
        print(f"An unexpected error occurred: {e}")
        traceback.print_exc()

User: "Måste man söka bygglov för flaggstång?"


Assistant: <b>
Ja, man måste söka bygglov för flaggstång inom detaljplanelagt område.</b>

User: "Måste man söka bygglov för altan?"


Assistant: <b>
Ja, man måste söka bygglov för altan om den är högre än 1,8 meter, placeras mindre än 4,5 meter från gränsen, eller om den kan betraktas som en volymökning eller väsentligt ändrar fasaden.</b>

User: "Måste man söka bygglov för carport?"


...

User: "Måste man söka bygglov för braskamin?"


Assistant: <b>
Ja, man måste söka bygglov för braskamin. En skorsten som är högre än 1,2 m kräver stag och plattform och införstartbesked. Om skorstenen är fyrkantig krävs både stag och plattform.</b>

User: "Vad är frankrikes huvudstad?"


Assistant: <b>
Paris.</b>

CPU times: user 477 ms, sys: 21.3 ms, total: 499 ms
Wall time: 37.7 s
